In [1]:
import os
import pandas as pd
import warnings
import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns

from tqdm import tqdm
from random import randint
from tqdm.auto import tqdm
from scipy.stats import entropy
import time
from sklearn.decomposition import PCA

plt.rcParams["figure.figsize"] = (20, 16)
warnings.simplefilter("ignore")



In [2]:
current_dataset = 3
name_directory = "CTU-13-Dataset"
path = os.path.join(name_directory,str(current_dataset))
os.listdir(path)

['botnet-capture-20110812-rbot.pcap', 'capture20110812.binetflow', 'README']

In [3]:
[i for i in os.listdir(path) if i.endswith(".binetflow") ]

['capture20110812.binetflow']

In [4]:
datafile_name = [i for i in os.listdir(path) if i.endswith(".binetflow") ][0]
df = pd.read_csv(os.path.join(path,datafile_name))
df

,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label
0,2011/08/12 15:25:56.021112,11.337043,tcp,195.68.34.68,52475,->,147.32.86.165,12114,SR_SA,0.0,0.0,11,824,606,flow=Background-TCP-Established
1,2011/08/12 15:29:25.508940,2.962470,tcp,147.32.86.58,1393,->,77.75.73.156,80,SR_A,0.0,0.0,3,182,122,flow=Background-TCP-Attempt
2,2011/08/12 15:30:21.101931,2.962828,tcp,201.54.33.206,2550,->,147.32.86.110,443,S_RA,0.0,0.0,4,240,120,flow=Background-TCP-Attempt
3,2011/08/12 15:37:08.317877,1.986249,tcp,221.134.221.114,8204,->,147.32.84.189,51413,S_RA,0.0,0.0,4,252,132,flow=Background-TCP-Attempt
4,2011/08/12 15:33:53.620500,767.978638,tcp,147.32.84.59,49156,->,147.32.80.7,80,SRPA_FSPA,0.0,0.0,14,3710,774,flow=Background-Established-cmpgw-CVUT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4710633,2011/08/15 10:13:26.395097,0.002210,udp,147.32.85.2,55572,<->,147.32.80.9,53,CON,0.0,0.0,2,505,78,flow=To-Background-UDP-CVUT-DNS-Server
4710634,2011/08/15 10:13:26.408351,0.000312,udp,147.32.85.2,61416,<->,147.32.80.9,53,CON,0.0,0.0,2,369,78,flow=To-Background-UDP-CVUT-DNS-Server
4710635,2011/08/15 10:13:26.412151,0.006138,tcp,147.32.85.2,49590,->,195.113.232.98,80,SPA_SPA,0.0,0.0,9,5206,295,flow=Background-TCP-Established
4710636,2011/08/15 10:13:26.427955,0.000549,udp,89.176.63.143,27520,<->,147.32.84.229,13363,CON,0.0,0.0,2,131,71,flow=Background-UDP-Established


In [5]:
def preprocess_df(df): 
    df['Sport'] = df['Sport'].fillna(-1)
    df['Dport'] = df['Dport'].fillna(-1)
    df["Backgroung_label"] = df.Label.str.contains("Background")
    df["Normal_label"] = df.Label.str.contains("Normal")
    df["Botnet_label"] = df.Label.str.contains("Botnet")
    df["StartTime"] = pd.to_datetime(df["StartTime"])
    return df
df = preprocess_df(df)
#df = df.iloc[0:100]

#  KGB


The entropy prediction algorithm presented by Lakhina et al. (2005) is based on the similar PCA-based traffic model than
Section 3.2.3, but it uses different features. It **aggregates the
traffic from the individual source IP addresses**, but instead of
traffic volumes, it predicts the **entropies of destination IP addresses**, **destination ports** and **source ports** 
over the set of context NetFlows for each source. The context space is
therefore three dimensional. 

Since the support of these distributions
is discrete and finite, they are simple histograms. Finally,
from these histograms we can easily calculate the entropy
by the usual formula H(x) = −sum(Pk.log Pk, foreach k)
where pk denotes the probability of kth bin.

In [6]:
def initial_context(df)
    df_for_for_entropies = df[["SrcAddr","DstAddr","Sport", "Dport","Label","Backgroung_label","Normal_label","Botnet_label"]]
    tmp_mesure = time.time_ns()
    base = 10
    entropy_agg =  lambda x : entropy(x.value_counts(), base=base)
    df_with_Entropies = df_for_for_entropies.groupby(by = "SrcAddr").agg({'DstAddr': entropy_agg,
                                                                            'Dport': entropy_agg,
                                                                             'Sport': entropy_agg})

    df_with_Entropies.rename(columns = {"DstAddr" : "H_DstAddr","Dport" : "H_Dport","Sport" : "H_Sport"}, inplace = True)
    df_with_Entropies
    print(f"Time required to compute aggregation : {np.round((time.time_ns() - tmp_mesure)/(10**9),2)} s")
    return df_with_Entropies

SyntaxError: invalid syntax (<ipython-input-6-a5dda3c90956>, line 1)



- For Lakhina Entropies

The complexity of PCT depends cubically on the
dimension of the feature vector x. Consequently,
the calculation of the PCT transformation might be
impossible, or at the time it will be calculated, it will
be no longer relevant.

To address these issues, the [8] simplified Lakhina’s detectors as follows: (a) it uses statistics aggregated only by
source IP addresses, and (b) it uses only latest five time
windows to build the model 1
. The feature vector in [8]
equals to

- For KGP

All entropies are calculated from all flows
observed during 5-minute long time windows.

The vector x
t
(ι) effectively describes user’s traffic at five
consecutive time windows t − 4, . . . , t. The entropies are
calculated iff the number of flows associated with the user
ι is higher than 1 (if there is only one flow originating at
given source IP, or ending at given destination IP, than all
entropies are equal to zero).

=> vecteur de taille 3 ou 15 (5*3 où on mesure l entropie sur chaque minute ????

For numerical stability, all vectors
yj corresponding to eigenvalues λj smaller than 10−6 are
discarded

In [ ]:
def compute_contexte_vector(df, window_time = 5, verbose = False, fillna= True):
    # window_time : the window_time which will be considered
    to_keep = ["StartTime","SrcAddr","DstAddr","Sport", "Dport","Label","Backgroung_label","Normal_label","Botnet_label"]
    df_for_entropies = df[to_keep]
    
    last_time = df_for_entropies.StartTime.max()
    time_min = last_time - pd.Timedelta(f"{window_time} minutes")
    df_for_entropies = df_for_entropies[df_for_entropies.StartTime>=time_min]

    #print(df_for_entropies)
    tmp_mesure = time.time_ns()
    base = 10
    entropy_agg =  lambda x : entropy(x.value_counts(), base=base)
    futur_df = pd.DataFrame({"SrcIP" : df_for_entropies.SrcAddr.unique()}).set_index("SrcIP")
    
    for i in range(window_time) :
        start = time_min + pd.Timedelta(f"{i} minutes")
        end = start + pd.Timedelta(f"{1} minutes")
        current_minute = df_for_entropies[(df_for_entropies.StartTime>=start) & (df_for_entropies.StartTime<end)]
    
        df_with_Entropies = current_minute.groupby(by = "SrcAddr").agg({'DstAddr': entropy_agg,
                                                                                'Dport': entropy_agg,
                                                                                 'Sport': entropy_agg})
        id_name = window_time-i-1
        df_with_Entropies.rename(columns = {"DstAddr" : f"H_DstAddr_t_{id_name}",
                                            "Dport" : f"H_Dport_t_{id_name}",
                                            "Sport" : f"H_Sport_t_{id_name}"}, inplace = True)
        futur_df = futur_df.merge(df_with_Entropies, left_index = True, right_index = True, how = "outer")
        
    if verbose : 
        print(f"Time required to compute aggregation : {np.round((time.time_ns() - tmp_mesure)/(10**9),2)} s")
    # Consequently, for successful detection, at least two flows per five minutes are needed.
    if fillna : # La j'ai un big doute
        futur_df = futur_df.fillna(0)  
    
    return futur_df


context = compute_contexte_vector(df, window_time = 5)
context

In [ ]:
def compute_pca_over_vector(df, drop_limit = 10**-6):
    
    pca = PCA()
    new_flows = pca.fit_transform(df)
    
    cov_matrix= pca.get_covariance()

    eigen_values_vectors = []
    for i, eigenvector in enumerate(pca.components_):

        eigen_value = np.dot(eigenvector.T, np.dot(cov_matrix, eigenvector))

        eigen_values_vectors.append((eigen_value,eigenvector))
        if eigen_value <  drop_limit :
            break

    new_df = pd.DataFrame(new_flows[:,:i+1],columns = [f"component_{j}" for j in range(i+1)])
    new_df.set_index(df.index, drop = True, inplace = True)
    
    
    return new_df, eigen_values_vectors
           
   
new_components, eigen_values_vectors,  = compute_pca_over_vector(context)    
new_components

In [ ]:
eigen_values_vectors[0]

In [ ]:
def compute_anomaly_score(row, eigen, major_components = True,k=1):

    if major_components : 
        end = k
        start = 1 
    else :
        end = len(row)
        start = k+1 
        
    value_sum = 0
    for j in range(start,end+1) :
        e_val, e_vec = eigen[j-1]
        #print(row.values)
        #print()
        val = np.dot(e_vec,np.array(row.values))
        
        val/=e_val
        val=val**2
        value_sum+=val
    
    return value_sum
compute_anomaly_score(context.iloc[0], eigen_values_vectors,major_components = False)

In [ ]:
res = context.apply(lambda x: compute_anomaly_score(x,eigen_values_vectors,major_components = True  ),
                                        axis = 1)
res

In [ ]:
#res.sort_values(ascending=False)

In [ ]:
#df[df.SrcAddr=="147.32.84.59"]

In [ ]:
#df[df.SrcAddr=="147.32.84.118"].sort_values("StartTime")

In [ ]:
def proceed_KGB(df, anomaly_threshold, window_time = 5, fill_na_ctxt = True,
                drop_limit_eigen = 10**-6,
                major_components = True,k=1
               ) :
    """
    df : obviously the df
    anomaly_threshold : an int
    
    window_time (opt) : int, the number of minutes considered in the df. we only consider the last window time in the df
    fill_na_ctxt (opt) : when computing the context, Some Ip source doesn't produce enough netflows to reach to ratio of 
    1 per minute, thus we can't compute their entropy. if fillna is set to True, fills theses nan values with 0
    
    drop_limit (opt) : Used to drop eigen values above a limit (default 10**-6: cf article). Supposed to be useful
    in order to maintain numerical stability.
    
    k (opt) : the number of component to consider as first components in the anomaly score (default = 1 : cf article)
    major_components (opt) : used if we want to use the methods of the principal components with the 
    highest variance (True) or the lowest (False)
    """
    
    context = compute_contexte_vector(df, window_time = window_time, fillna = fill_na_ctxt)
    
    new_components, eigen_values_vectors = compute_pca_over_vector(context, drop_limit = drop_limit_eigen)  
    
    
    res = context.apply(lambda x: compute_anomaly_score(x,eigen_values_vectors,major_components = major_components, k=k  ),
                                        axis = 1)
    #return res.sort_values(ascending=False)
    
    return [ index for index, row in res.iteritems() if row>anomaly_threshold]



In [ ]:
proceed_KGB(df, 2000 ) 

No Python documentation found for 'proceed_KGB'.
Use help() to get the interactive help utility.
Use help(str) for help on the str class.

